In [2]:
pip install langchain_community

  Obtaining dependency information for tenacity!=8.4.0,<9.0.0,>=8.1.0 from https://files.pythonhosted.org/packages/d2/3f/8ba87d9e287b9d385a02a7114ddcef61b26f86411e121c9003eb509a1773/tenacity-8.5.0-py3-none-any.whl.metadata
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response=llm.invoke("second person to land on moon was...")
print(response.content)

The second person to land on the moon was Edwin "Buzz" Aldrin. He stepped onto the lunar surface on July 20, 1969, during the Apollo 11 mission, following Neil Armstrong, who was the first person to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://jobs.klimb.io/petpooja/650d96e49311af435b875650/?source=careers")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.




















Petpooja, Intern - Data Science, Ahmedabad, Gujarat, India






































									
				        				Intern - Data Science
									
			        


Ahmedabad, Gujarat, India 

Internship, null months

Data Science
0 - 1 years




						View All Jobs
					



								Apply
								



					About us
				 






Share this job





Facebook
Twitter
LinkedIn
Whatsapp



Job Title:


							Intern - Data Science
						



Job Description:


Skills Required:Basic PythonMachine LearningDeep LearningBasic Understanding of TransformersBasic Understanding of LLMsBasic Understanding of Prompt WritingBasic Understanding of RAGsOne or more projects in Transformers, LLMs, RAGs, etc




Experience Range:
0 - 1 years


Educational Qualifications:

				
				
						
							Any graduation,
						
						
					
						
						
					
				
				

				

				
						
						
					 
				
				
				
						
						
					 
				

				


Job Responsibilities:




Working on data clean

In [13]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
        """
        ###SCRAPED TEXT FROM WEBSITE
        {page_data}
        ###INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing
        following keys:'role','experience','skills' and 'description'.
        Only return valid JSON.
        ###VALID JSON (NO PREAMBLE )
        """
)
chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [22]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_response=json_parser.parse(res.content)
json_response=json_response[0]
type(json_response)

dict

In [31]:
job_des=json_response
job_des

{'role': 'Intern - Data Science',
 'experience': '0 - 1 years',
 'skills': ['Python',
  'Machine Learning',
  'Deep Learning',
  'LLM',
  'Transformers',
  'Prompt Writing',
  'RAG'],
 'description': "Working on data cleansing to start with. Assist the team of Data Scientist with their business requirements. Assisting, your team lead with your knowledge on Python, Machine Learning and Deep Learning. It's a 6 days working internship for 6 months. Where you would be getting 10000 Rs stipend monthly."}

In [25]:
import pandas as pd
df=pd.read_csv('portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [30]:
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [27]:
import chromadb
import uuid
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row['Techstack'],metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [33]:
prompt_email=PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Harshdeep, a business development executive at STechs. STechs is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client company's appropriate authorities regarding the job mentioned above describing the capability of STechs 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase STechs's portfolio: {link_list}
        Remember you are Harshdeep, BDE at STechs. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job_des), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Your Business Needs

Dear Hiring Manager,

I hope this email finds you well. I am Harshdeep, a Business Development Executive at STechs, an AI & Software Consulting company dedicated to empowering businesses with tailored solutions. We came across your job posting for an Intern - Data Science role, and I am excited to introduce our company as a potential partner to fulfill your data science needs.

At STechs, we have a proven track record of delivering scalable, optimized, and efficient solutions to various enterprises. Our team of expert data scientists and engineers has extensive experience in Python, Machine Learning, Deep Learning, and other relevant technologies. We are confident that our capabilities align with your requirements and can support your team in achieving their business objectives.

Our portfolio showcases our expertise in Machine Learning and Python, which can be seen in the following projects:

* https://example.com/ml-pyth